In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import jieba
import os
from torch.nn import init
from torchtext import data
from torchtext.vocab import Vectors
import time
import math

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device=torch.device('cpu')

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# data processing

In [3]:
# 分词
def tokenizer(text): 
    return [word for word in jieba.lcut(text) if word not in stop_words]

In [4]:
# 去停用词
def get_stop_words():
    file_object = open('stopwords.txt',encoding='utf-8')
    stop_words = []
    for line in file_object.readlines():
        line = line[:-1]    #删除换行符 （？）
        line = line.strip() #删除换行符 空格
        stop_words.append(line)
    return stop_words

stop_words = get_stop_words()  # 加载停用词表

In [5]:
# stop_words[:10]

In [6]:
text = data.Field(sequential=True, #？
                  lower=True,
                  tokenize=tokenizer,
                  fix_length=500,  #一句话的长度 不够会补全<pad>
                  stop_words=stop_words) #定义处理方法
label = data.Field(sequential=False)

In [7]:
train, val = data.TabularDataset.splits( #TabularDataset适合处理csv tsv
    path='',
    skip_header=True,
    train='large_data/10.tsv',
    validation='large_data/1.tsv',
    #train='large_data/1.tsv',
    #validation='large_data/1.tsv',
    format='tsv',
    fields=[ ('label', label), ('text', text)],
)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\92798\AppData\Local\Temp\jieba.cache
Loading model cost 0.612 seconds.
Prefix dict has been built succesfully.


In [8]:
# print(train[2].text) #第二行的内容
# print(train[2].label) #第二行的标签

# print(train[5].__dict__.keys())
# print(train[5])

In [9]:
#加载Google训练的词向量
# import gensim
# model = gensim.models.KeyedVectors.load_word2vec_format('myvector.vector', binary=False)

In [10]:
# cache = '.vector_cache' #?
# if not os.path.exists(cache):
#     os.mkdir(cache)
vectors = Vectors("word_embedding/chinese_wiki_embeding20000.txt")
# 指定Vector缺失值的初始化方式，没有命中的token的初始化方式
#vectors.unk_init = nn.init.xavier_uniform_

text.build_vocab(train, val, vectors=vectors)#加入测试集的vertor

In [11]:
#text.build_vocab(train, val, vectors=Vectors(name='data/myvector.vector'))#加入测试集的vertor
label.build_vocab(train, val)
embedding_dim = text.vocab.vectors.size()[-1]
vectors = text.vocab.vectors

In [12]:
# text.vocab.freqs.most_common(10)
# print(text.vocab.vectors.shape)

In [13]:
batch_size=64
train_iter, val_iter = data.Iterator.splits(
            (train, val),
            sort_key=lambda x: len(x.text),
            batch_sizes=(batch_size, len(val)), # 训练集设置batch_size,验证集整个集合用于测试
            device=device
    )

vocab_size = len(text.vocab)
label_num = len(label.vocab)

In [14]:
batch = next(iter(train_iter))
data = batch.label
print(data)
# print(batch.text.shape)
# print(batch.text)


tensor([ 9,  2,  3,  1,  6,  9,  5,  8,  6,  7,  7,  9,  6,  5,  6,  7,  6,  4,
         4,  5,  8,  8, 10,  3,  8,  2,  3,  6,  6,  2,  3,  1,  1, 10,  6,  6,
         6,  5,  5,  5,  5,  2,  5,  5,  3,  1,  9,  2,  5,  2,  1,  6,  7,  5,
         7,  6,  1,  6,  2,  7, 10,  4,  8,  8], device='cuda:0')


In [15]:
print(label.vocab.stoi)

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x000002A794897C08>>, {'<unk>': 0, '娱乐': 1, '科技': 2, '时政': 3, '教育': 4, '体育': 5, '游戏': 6, '家居': 7, '财经': 8, '时尚': 9, '房产': 10})


# model

In [16]:
class BiRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_hiddens, num_layers,n_filter,filter_size):
        super(BiRNN, self).__init__()
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)  # embedding之后的shape: torch.Size([128, 100, 100])
        self.word_embeddings = self.word_embeddings.from_pretrained(vectors, freeze=False)
        # bidirectional设为True即得到双向循环神经网络
        self.encoder = nn.LSTM(input_size=embedding_dim, 
                                hidden_size=num_hiddens, 
                                num_layers=num_layers,
                                batch_first=True,
                                bidirectional=True)
        
        #第一个CNN
        self.conv1=nn.Sequential(
            nn.Conv2d(in_channels=1,
                     out_channels=n_filter,
                     kernel_size=(filter_size[0],300), 
                     stride=1,
                     padding=0), # 输出 [498,1]
            nn.BatchNorm2d(n_filter),
            nn.ReLU()
        )
        
        
        self.conv2=nn.Sequential(
            nn.Conv2d(n_filter,n_filter*2,(filter_size[1],1),1,0), #[245,1]
            nn.BatchNorm2d(n_filter*2),
            nn.ReLU())
        
        self.conv3=nn.Sequential(
            nn.Conv2d(n_filter*2,n_filter*4,(filter_size[2],1),1,0), #[120,1]   
            nn.BatchNorm2d(n_filter*4),
            nn.ReLU())
        self.conv4=nn.Sequential(
            nn.Conv2d(n_filter*4,n_filter*4,(filter_size[2],1),1,0), #[58,1]   
            nn.BatchNorm2d(n_filter*4),
            nn.ReLU())
        
        #第二个CNN
        self.conv5=nn.Sequential(
            nn.Conv2d(in_channels=1,
                     out_channels=n_filter,
                     kernel_size=(filter_size[3],300), 
                     stride=1,
                     padding=0), # 输出 [497,1]
            nn.BatchNorm2d(n_filter),
            nn.ReLU()
        )
        
        self.conv6=nn.Sequential(
            nn.Conv2d(n_filter,n_filter*2,(filter_size[4],1),1,0), #[246,1]
            nn.BatchNorm2d(n_filter*2),
            nn.ReLU())
        
        self.conv7=nn.Sequential(
            nn.Conv2d(n_filter*2,n_filter*4,(filter_size[5],1),1,0), #[122,1]
            nn.BatchNorm2d(n_filter*4),
            nn.ReLU())
        self.conv8=nn.Sequential(
            nn.Conv2d(n_filter*4,n_filter*4,(filter_size[5],1),1,0), #[60,1]   
            nn.BatchNorm2d(n_filter*4),
            nn.ReLU())
        
        
        self.pool1=nn.MaxPool2d(
            kernel_size=(2,1),  #[8,1]
            stride=(2,1),
            padding=0)
        
        self.pool2=nn.MaxPool2d(
            kernel_size=(29,1),  #[32,7,1]
            stride=(29,1),
            padding=0)
        self.pool3=nn.MaxPool2d(
            kernel_size=(20,1),  #[32,7,1]
            stride=(20,1),
            padding=0)
        self.decoder=nn.Linear(800,10)
        
        
    def attention(self,query,keys,values, dropout=None):
        d_k = keys.size(-1)  # 64=d_k
#         print("query",query.shape)
        scores = torch.matmul(query, keys) /math.sqrt(d_k)
#        print("scores",scores.shape)
        p_attn = F.softmax(scores, dim=-1)
#        print("p_attn",p_attn.shape)
        # 对scores的最后一个维度执行softmax，得到的还是一个tensor,
        # (30, 8, 10, 11)
        if dropout is not None:
            p_attn = dropout(p_attn)  # 执行一次dropout
        fea  = torch.matmul(p_attn, values.transpose(-2, -1))
        return fea, p_attn
    
    def forward(self, inputs):
        # inputs的形状是(批量大小, 词数)，因为LSTM需要将序列长度(seq_len)作为第一维，所以将输入转置后
        # 再提取词特征，输出形状为(词数, 批量大小, 词向量维度)
        embeddings = self.word_embeddings(inputs)
        fast_embeddings=F.avg_pool2d(embeddings,(2,1),stride=1)
        
        #对embedding进行Lstm
        # outputs形状是(词数, 批量大小, 2 * 隐藏单元个数)
        outputs, _ = self.encoder(embeddings) # output, (h, c)
        feature_lstm=outputs
        
        
        #对embedding进行CNN
        #print(outputs.shape)
        outputs_pred=outputs.unsqueeze(1) #转换后的矩阵为[batch,seq_len,2*hidden_state] [128，100，2*64]
        #print(feature1.shape)
        feature1=self.conv1(embeddings.unsqueeze(1)) #[128,32,490,1]
        feature1=self.pool1(feature1)
        feature1=self.conv2(feature1)
        feature1=self.pool1(feature1)
        feature1=self.conv3(feature1)
        feature1=self.pool1(feature1)
        feature1=self.conv4(feature1)
        feature1=self.pool2(feature1).squeeze(2) #输出 [batch,层数, H,W] #[128,32,7]
        feature1=feature1.reshape(-1,feature1.shape[1]*feature1.shape[2])
        feature2=self.conv5(embeddings.unsqueeze(1)) #[128,32,480,1]
        feature2=self.pool1(feature2)
        feature2=self.conv6(feature2)
        feature2=self.pool1(feature2)
        feature2=self.conv7(feature2)
        feature2=self.pool1(feature2)
        feature2=self.conv8(feature2)
        feature2=self.pool3(feature2).squeeze(2) #输出 [batch,层数, H,W] #[128,32,8])
        feature2=feature2.reshape(-1,feature2.shape[1]*feature2.shape[2]) #feature2=[batch_size, 32*8=]
        feature_cnn=torch.cat((feature1,feature2),dim=1).unsqueeze(1)
        feature_cnn=F.dropout(feature_cnn,p=0.5,training=self.training)
        print("cnn",feature_cnn.shape)
        print("lstm",feature_lstm.shape)
        feature_ft=F.avg_pool2d(fast_embeddings,(embeddings.shape[1]-1,1)).squeeze(1)  #feature_ft = [batch size, embedding_dim]
        
        fea_attn,attnMatrix=self.attention(feature_cnn,feature_lstm,feature_lstm)
        fea_attn=fea_attn.reshape(fea_attn.shape[0],-1)
    
        Fea=torch.cat((fea_attn,feature_ft),dim=1)
        print("fea_attn",fea_attn.shape)
        print("feature_ft",feature_ft.shape)
        outs = self.decoder(Fea)
        return outs

In [17]:
embedding_dim, num_hiddens, num_layers = 300, 64, 1
n_filter=25
filter_size=[5,4,3,4,3,2]
net = BiRNN(vocab_size, embedding_dim, num_hiddens, num_layers,n_filter,filter_size)
#print(net)

# train

In [18]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    with torch.no_grad():
        for batch_idx, batch in enumerate(train_iter):
            X, y = batch.text, batch.label
            X = X.permute(1, 0)
#             print(y.data)
            y.data.sub_(1)  #X转置 y要减1：计算习惯，0代表第一类，1代表第二类
            if isinstance(net, torch.nn.Module):
                net.eval() # 评估模式, 这会关闭dropout
                acc_sum += (net(X).argmax(dim=1) == y).float().sum().item()
                net.train() # 改回训练模式
            else: # 自定义的模型, 不考虑GPU
                if('is_training' in net.__code__.co_varnames): # 如果有is_training这个参数
                    # 将is_training设置成False
                    acc_sum += (net(X, is_training=False).argmax(dim=1) == y).float().sum().item() 
                else:
                    acc_sum += (net(X).argmax(dim=1) == y).float().sum().item() 
            n += y.shape[0]
    return acc_sum / n

In [19]:
def train(train_iter, test_iter, net, loss, optimizer, num_epochs):
    batch_count = 0
    epoch_list_train=[]
    epoch_list_val=[]
    epoch_list_loss=[]
    first_loss=0
    flag=1
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for batch_idx, batch in enumerate(train_iter):
            #print("batch:"+str(batch_idx))
            X, y = batch.text, batch.label
            X = X.permute(1, 0)
            y.data.sub_(1)  #X转置 y为啥要减1
            y_hat = net(X)
            l = loss(y_hat, y)
            if flag==1:
                flag=0
                first_loss=l.item()
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().item()
            n += y.shape[0]
            batch_count += 1
        test_acc = evaluate_accuracy(test_iter, net)
        print(
            'epoch %d, loss %.4f, train acc %.3f, test acc %.3f, time %.1f sec'
            % (epoch + 1, train_l_sum / batch_count, train_acc_sum / n,
               test_acc, time.time() - start))
        epoch_list_loss.append(train_l_sum / batch_count)
        epoch_list_train.append(train_acc_sum / n)
        epoch_list_val.append(test_acc)
    return epoch_list_loss,epoch_list_train,epoch_list_val,first_loss

In [20]:
lr, num_epochs = 0.01, 1
net=net.to(device)
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
loss = nn.CrossEntropyLoss().to(device)
epoch_list_loss,epoch_list_train,epoch_list_val,first_loss=train(train_iter, val_iter, net, loss, optimizer, num_epochs)ii

cnn torch.Size([64, 1, 500])
lstm torch.Size([64, 500, 128])
fea_attn torch.Size([64, 500])
feature_ft torch.Size([64, 300])
cnn torch.Size([64, 1, 500])
lstm torch.Size([64, 500, 128])
fea_attn torch.Size([64, 500])
feature_ft torch.Size([64, 300])
cnn torch.Size([64, 1, 500])
lstm torch.Size([64, 500, 128])
fea_attn torch.Size([64, 500])
feature_ft torch.Size([64, 300])
cnn torch.Size([64, 1, 500])
lstm torch.Size([64, 500, 128])
fea_attn torch.Size([64, 500])
feature_ft torch.Size([64, 300])
cnn torch.Size([64, 1, 500])
lstm torch.Size([64, 500, 128])
fea_attn torch.Size([64, 500])
feature_ft torch.Size([64, 300])
cnn torch.Size([64, 1, 500])
lstm torch.Size([64, 500, 128])
fea_attn torch.Size([64, 500])
feature_ft torch.Size([64, 300])
cnn torch.Size([64, 1, 500])
lstm torch.Size([64, 500, 128])
fea_attn torch.Size([64, 500])
feature_ft torch.Size([64, 300])
cnn torch.Size([64, 1, 500])
lstm torch.Size([64, 500, 128])
fea_attn torch.Size([64, 500])
feature_ft torch.Size([64, 300])


KeyboardInterrupt: 

In [21]:
import time,csv
 
localtime = time.asctime( time.localtime(time.time()) )
result=epoch_list_loss+epoch_list_train+epoch_list_val

In [22]:
filepath="result/"+str(localtime).replace(":"," ")+".txt"
with open(filepath,'w',encoding='utf-8') as f_output:
    tsv_output=csv.writer(f_output,delimiter=',')
    tsv_output.writerow(result)

In [23]:
print(first_loss)

2.297832727432251


In [24]:
# test_acc = evaluate_accuracy(val_iter, net)
# print(test_acc)

In [25]:
_20

NameError: name '_20' is not defined